# nn 모듈

## PyTorch: nn

- Computational graph와 `autograd`는 너무 low-level이라서 큰 신경망에 사용하기는 어려움
- PyTorch에서는 computational graph를 higher-level로 추상화하는 패키지로 `nn`을 사용

### Example
- nn 패키지를 사용해 2계층 신경망을 구성하기

In [1]:
"""
네트워크 구성
"""

# N : batch size
# H : hidden layer의 차원
# D_in : 입력의 차원
# D_out : 출력의 차원
N, H, D_in, D_out = 64, 100, 1000, 10

# learning rate
lr = 1e-4

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [3]:
x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

# nn.Sequential은 각각의 Module들을 순차적으로 적용해서 출력을 생성
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device)

# mean squared loss (squared L2 norm)
# reduction="none" : 벡터 형태
# reduction="mean" : 벡터 값들에서 mean 해준 결과, torch.nn.MSELoss(reduction="none").mean()과 동일
# reduction="mean" : 벡터 값들 모두 더한 결과, torch.nn.MSELoss(reduction="none").sum()과 동일
loss_fn = torch.nn.MSELoss(reduction="sum")

for t in range(500):
    y_pred = model(x)

    # MSE Loss 계산
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # gradient를 0으로 비워줌
    model.zero_grad()

    # Backprop
    loss.backward()

    # Update
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad
            
            
print("")
print((y_pred - y).sum().item())

99 2.448528289794922
199 0.044219970703125
299 0.0019115060567855835
399 0.00011589543282752857
499 8.144764251483139e-06

0.000228236080147326


## PyTorch: optim

- PyTorch의 `optim` 패키지는 최적화 알고리즘에 대한 아이디어를 추상화하고 일반적으로 사용하는 최적화 알고리즘의 구현체(implementation)를 제공
    - AdaGrad, RMSProp, Adam 등의 Optimizer 제공

### Example
- 앞의 예제에 `optim` 패키지가 제공하는 Adam 알고리즘을 적용

In [4]:
"""
네트워크 구성
"""

# N : batch size
# H : hidden layer의 차원
# D_in : 입력의 차원
# D_out : 출력의 차원
N, H, D_in, D_out = 64, 100, 1000, 10

# learning rate
lr = 1e-4

In [5]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:
x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

# nn.Sequential은 각각의 Module들을 순차적으로 적용해서 출력을 생성
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out)
)
model.to(device)

# mean squared loss (squared L2 norm)
# reduction="none" : 벡터 형태
# reduction="mean" : 벡터 값들에서 mean 해준 결과, torch.nn.MSELoss(reduction="none").mean()과 동일
# reduction="mean" : 벡터 값들 모두 더한 결과, torch.nn.MSELoss(reduction="none").sum()과 동일
loss_fn = torch.nn.MSELoss(reduction="sum")

# Adam optimizer 사용
# update될 파라미터와 learning rate를 인자로 전달
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for t in range(500):
    y_pred = model(x)

    # MSE Loss 계산
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # gradient를 0으로 비워줌
    model.zero_grad()

    # Backprop
    loss.backward()

    # Update
    optimizer.step()
            

print("")
print((y_pred - y).sum().item())

99 51.018287658691406
199 0.6328063607215881
299 0.0012006131000816822
399 1.123887727771944e-06
499 6.04595484787751e-10

-2.612592652440071e-06


## PyTorch: 사용자 정의 nn.Module

- Sequential하지 않은 복잡한 모델을 구성해야 할 때는 `nn.Module`의 서브클래스로 모듈을 정의할 수 있음
    - 서브클래스의 메서드로 `forward()` 를 정의해야 함

### Example
- 2계층 신경망을 `nn.Module`의 서브클래스로 구현

In [7]:
"""
네트워크 구성
"""

# N : batch size
# H : hidden layer의 차원
# D_in : 입력의 차원
# D_out : 출력의 차원
N, H, D_in, D_out = 64, 100, 1000, 10

# learning rate
lr = 1e-4

In [8]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [9]:
class TwoLayerNet(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):
        """
        생성자에서 layer를 정의하고, 멤버 변수로 지정
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.linear2 = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        """
        Forward pass
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred
    

# 모델 정의
model = TwoLayerNet(D_in, H, D_out)
model.to(device)

TwoLayerNet(
  (linear1): Linear(in_features=1000, out_features=100, bias=True)
  (linear2): Linear(in_features=100, out_features=10, bias=True)
)

In [10]:
x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

loss_fn = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    model.zero_grad()
    loss.backward()
    optimizer.step()
            

print("")
print((y_pred - y).sum().item())

99 59.13008117675781
199 0.9483639597892761
299 0.0025306076277047396
399 2.036195837717969e-06
499 4.627082328401144e-10

-6.194633897393942e-06


## PyTorch: 제어 흐름(Control Flow) + 가중치 공유(Weight Sharing)

- Forward pass에서 동일한 Module을 여러번 재사용 가능하므로, 내부 layer들간의 가중치 공유를 구현 가능

### Example 1
- 동적으로 그래프를 생성하고, 가중치를 재사용하는 모델 구현

In [11]:
import torch
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

# learning rate
lr = 1e-4

x = torch.randn(N, D_in, device=device, dtype=torch.float)
y = torch.randn(N, D_out, device=device, dtype=torch.float)

In [12]:
class DynamicNet(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        
        # 각 forward pass마다 동적으로 graph를 구성
        # 동일한 Module(middle_linear)을 여러번 재사용 가능
        
        count = 0 # 동적으로 생성되는 graph 수 count
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
            count += 1
            
        print(count)
        y_pred = self.output_linear(h_relu)
        
        return y_pred

In [13]:
model = DynamicNet(D_in, H, D_out)
model.to(device)

# forward pass에서 동적으로 graph를 생성하는지 확인
for t in range(10):
    y_pred = model(x)

1
3
1
3
3
0
3
2
2
2


### Example 2
- 앞서 구현한 모델 학습

In [14]:
class DynamicNet(torch.nn.Module):
    
    def __init__(self, D_in, H, D_out):
        super(DynamicNet, self).__init__()
        self.input_linear = torch.nn.Linear(D_in, H)
        self.middle_linear = torch.nn.Linear(H, H)
        self.output_linear = torch.nn.Linear(H, D_out)
        
    def forward(self, x):
        h_relu = self.input_linear(x).clamp(min=0)
        
        # 각 forward pass마다 동적으로 graph를 구성
        # 동일한 Module(middle_linear)을 여러번 재사용 가능
        for _ in range(random.randint(0, 3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
            
        y_pred = self.output_linear(h_relu)
        
        return y_pred
    
    
model = DynamicNet(D_in, H, D_out)
model.to(device)

DynamicNet(
  (input_linear): Linear(in_features=1000, out_features=100, bias=True)
  (middle_linear): Linear(in_features=100, out_features=100, bias=True)
  (output_linear): Linear(in_features=100, out_features=10, bias=True)
)

In [15]:
loss_fn = torch.nn.MSELoss(reduction="sum")

# SGD with momentum=0.9
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    model.zero_grad()
    loss.backward()
    optimizer.step()
            

print("")
print((y_pred - y).sum().item())

99 20.132814407348633
199 1.2074705362319946
299 0.23595987260341644
399 1.9188899993896484
499 0.12006747722625732

-0.6344144940376282
